# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Big Data** </center>
---
### <center> **Spring 2025** </center>
---
### <center> **Streamer Structrure for final Proyect** </center>
### <center> **Par de Foraneos** </center>


In [1]:
import findspark
findspark.init()


## Spark Session creation


In [2]:
from pyspark.sql import SparkSession

SPARK_SERVER = {'Konrad': '2453c3db49e4',
                'Aaron' : 'a5ab6bdab4b3'}
KAFKA_SERVER = {'Konrad': '4c63f45c41b4:9093',
                'Aaron' : '69b1b3611d90:9093'}
current_user = 'Konrad'

spark = SparkSession.builder \
    .appName("reader-Kafka") \
    .master("spark://{}:7077".format(SPARK_SERVER[current_user])) \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()
    
sc = spark.sparkContext

spark.conf.set("spark.sql.shuffle.partitions", "5")

:: loading settings :: url = jar:file:/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0038e828-9a61-422f-9e41-203e0ddd9e11;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;3.5.4 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;3.5.4 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.scala-lang.modules#scala-parallel-collections_2.13;1.0.4 in central
	found org.apache.commons#commons-pool2;2.11.1 in centr

## Kafka Stream creation

## Transform binary data into string

### Watermarking to handle late arrival events

### Sink configuration

In [3]:
import os
import glob

In [4]:
files = glob.glob("/home/jovyan/notebooks/data/final_project_ParDeForaneos/output0/*.parquet")

In [5]:
df0 = spark.read.parquet(files[0])
for i in range(1,len(files)):
    df1 = spark.read.parquet(files[i])
    df0 = df0.union(df1)

In [6]:
a=df0.orderBy("timestamp", ascending=True).dropDuplicates(["timestamp"])


In [7]:
a.show()

+-------------------+-------+---------+---------+---------+---------+
|          timestamp|company|     open|     high|      low|    close|
+-------------------+-------+---------+---------+---------+---------+
|2025-05-13 20:15:00|    CAT|342.93753|343.20148|342.93753|343.20148|
|2025-05-13 20:20:00|    CAT|345.62918|345.77954| 345.6173| 345.6173|
|2025-05-13 20:25:00|    CAT| 345.5698|345.66568|345.42758| 345.5017|
|2025-05-13 20:30:00|    CAT|343.89902|344.33096| 343.7308| 344.3125|
|2025-05-13 20:35:00|    CAT|344.30045|344.30045|344.06238|344.28622|
|2025-05-13 20:40:00|    CAT|344.26022| 344.7602|344.13525| 344.7602|
|2025-05-13 20:45:00|    CAT|344.74054|345.38028|344.74054|345.35986|
|2025-05-13 20:50:00|    CAT|345.40012|345.40012|344.73892|344.76483|
|2025-05-13 20:55:00|    CAT| 344.6523|344.88815|344.62592| 344.8439|
|2025-05-13 21:00:00|    CAT|344.83353|345.12054| 344.6902| 344.9121|
|2025-05-13 21:05:00|    CAT|344.94507|344.94507|  344.495|344.85913|
|2025-05-13 21:10:00

In [8]:
from foraneos.utils_proyectofinal_copy import calc_techincal_indicators

In [9]:
pandas_df = a.toPandas()
custom_resampler = calc_techincal_indicators(pandas_df)

Calculating technical indicators...
Creating lag feature for lag 1...
Creating lag feature for lag 2...
Creating lag feature for lag 3...
Creating lag feature for lag 4...
Creating lag feature for lag 5...
            timestamp company        open        high         low       close  \
0 2025-05-13 20:15:00     CAT  342.937531  343.201477  342.937531  343.201477   
1 2025-05-13 20:20:00     CAT  345.629181  345.779541  345.617310  345.617310   
2 2025-05-13 20:25:00     CAT  345.569794  345.665680  345.427582  345.501709   
3 2025-05-13 20:30:00     CAT  343.899017  344.330963  343.730804  344.312500   
4 2025-05-13 20:35:00     CAT  344.300446  344.300446  344.062378  344.286224   

   williams_r  rsi  ultimate_osc  ema  close_lag_1  close_lag_2  close_lag_3  \
0         NaN  NaN           NaN  NaN          NaN          NaN          NaN   
1         NaN  NaN           NaN  NaN   343.201477          NaN          NaN   
2         NaN  NaN           NaN  NaN   345.617310   343.201477    

In [ ]:
spark_df = spark.createDataFrame(custom_resampler)

In [ ]:
spark_df.show()

In [12]:
sc.stop()

25/05/13 21:16:46 WARN JavaUtils: Attempt to delete using native Unix OS command failed for path = /tmp/spark-c92e3eb1-1171-4496-bb94-87ff512079b5/pyspark-2db53aa6-0088-42bf-9c54-6ad15394ee3f. Falling back to Java IO way
java.io.IOException: Failed to delete: /tmp/spark-c92e3eb1-1171-4496-bb94-87ff512079b5/pyspark-2db53aa6-0088-42bf-9c54-6ad15394ee3f
	at org.apache.spark.network.util.JavaUtils.deleteRecursivelyUsingUnixNative(JavaUtils.java:174)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:109)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:90)
	at org.apache.spark.util.SparkFileUtils.deleteRecursively(SparkFileUtils.scala:121)
	at org.apache.spark.util.SparkFileUtils.deleteRecursively$(SparkFileUtils.scala:120)
	at org.apache.spark.util.Utils$.deleteRecursively(Utils.scala:1126)
	at org.apache.spark.util.ShutdownHookManager$.$anonfun$new$4(ShutdownHookManager.scala:65)
	at org.apache.spark.util.ShutdownHookManager$.$anonfun